In [ ]:
MYTOOLBOXPATH='../../../'
from jupyterlab_h5web import H5Web
from IPython.display import Image
import sys, os
import h5py, ase
import numpy as np
sys.path.append(MYTOOLBOXPATH + '/code')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-parmsetup/src/python')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-transcoder/src/python')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-autoreporter/src/python')
print('Current working directory is')
print(os.getcwd())

In [ ]:
from tools.transcoder_guru import ParmsetupTranscoder, TranscodingTask
from tools.ranger_guru import ParmsetupRanger, ApplyExistentRanging, MolecularIonSearch
from wizard.ranger_report import AutoReporterRanger

In [ ]:
# specify the location where you have your data on the system
# testing without NeXus
MY_MEASURED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_data/usa_portland_wang'
MY_PROCESSED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_analyses/usa_portland_wang'
# testing with NeXus
# MY_MEASURED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_data/ger_berlin_kuehbach_fairmat'

# specify disjoint identifier with which all config and results files for this analysis will be tagged.
jobids = [636502001]
JOBIDS = []
for jobid in jobids:
    assert isinstance(jobid, int), 'identifier needs to be an unsigned integer !'
    assert jobid != 0, 'identifier must not be 0 !'
    assert jobid <= np.iinfo(np.uint32).max, 'identifier needs to be on interval [1, 4294967295]'
    JOBIDS.append(str(jobid))
print(JOBIDS)

In [ ]:
TRANSCODER_CONFIG = {}
RECONSTRUCTION_AND_RANGING = {}
RECONSTRUCTION_AND_RANGING[jobids[0]] = ('R31_06365-v02.pos', 'R31_06365-v02.rrng')

for jobid in jobids:
    JOBID = str(jobid)
    transcoder = ParmsetupTranscoder()

    TRANSCODER_CONFIG[jobid] = transcoder.load_reconstruction_and_ranging(
        working_directory='',
        reconstructed_dataset=MY_MEASURED_DATA_PATH + '/' + RECONSTRUCTION_AND_RANGING[jobid][0],
        ranging_definitions=MY_MEASURED_DATA_PATH + '/' + RECONSTRUCTION_AND_RANGING[jobid][1],
        jobid=jobid)

print(TRANSCODER_CONFIG)

In [ ]:
H5Web(TRANSCODER_CONFIG[jobid])

In [ ]:
# execute paraprobe-transcoder Python tool
TRANSCODER_RESULTS = {}
for jobid in jobids:
    transcoder = ParaprobeTranscoder(TRANSCODER_CONFIG[jobid])
    TRANSCODER_RESULTS[jobid] = transcoder.execute()
print(TRANSCODER_RESULTS)

In [ ]:
get_file_size(TRANSCODER_RESULTS[jobid])
H5Web(TRANSCODER_RESULTS[jobid])

In [ ]:
RANGER_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    ranger = ParmsetupRanger()
    RANGER_CONFIG[jobid] = ranger.apply_existent_ranging(
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid)
print(RANGER_CONFIG)

In [ ]:
get_file_size(RANGER_RESULTS[jobid])
H5Web(RANGER_RESULTS[jobid])

In [ ]:
for jobid in jobids:
    JOBID = str(jobid)
    print(JOBID)
    ranger_report = AutoReporterRanger(RANGER_RESULTS[jobid], JOBID)
    ranger_report.get_summary()